In [1]:
from pyvis import network as net
import networkx as nx 
from pyvis.network import Network
import streamlit as st
from IPython.display import display, HTML
#from IPython.core.display import display, HTML

In [70]:
import csv
import networkx as nx
import os

def create_graph_from_csv(csv_path, conditions):
    # 读取 CSV 文件
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # 为每个条件创建一个空图
        graphs = {condition: nx.Graph() for condition in conditions}
        # 创建一个总的图
        total_graph = nx.Graph()
        
        # 解析 CSV 文件中的每一行
        for row in reader:
            nodes = row['ROW_NAMES']
            node1, node2 = nodes.split('_')
            
            # 按条件添加边到各自的图
            for condition in conditions:
                if row[condition]:
                    graphs[condition].add_edge(node1, node2)
                    total_graph.add_edge(node1, node2)
    
    return graphs, total_graph

def save_graphs_to_graphml(graphs, total_graph, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for condition, graph in graphs.items():
        output_path = os.path.join(output_dir, f"{condition}.graphml")
        nx.write_graphml(graph, output_path)
        print(f"Graph for {condition} saved to {output_path}")
    
    total_output_path = os.path.join(output_dir, "Total.graphml")
    nx.write_graphml(total_graph, total_output_path)
    print(f"Total graph saved to {total_output_path}")

In [71]:
csv_path = 'data/Total_Network_Classes.csv'
conditions = ['Independent', 'HM_Enhanced', 'Neu_Enhanced', 'F_Enhanced', 'S_Enhanced', 'FS_Enhanced',
              'HM_Suppressed','Neu_Suppressed','F_Suppressed','S_Suppressed','FS_Suppressed']
output_dir = 'data/graphml'

graphs, total_graph = create_graph_from_csv(csv_path, conditions)
save_graphs_to_graphml(graphs, total_graph, output_dir)

Graph for Independent saved to data/graphml/Independent.graphml
Graph for HM_Enhanced saved to data/graphml/HM_Enhanced.graphml
Graph for Neu_Enhanced saved to data/graphml/Neu_Enhanced.graphml
Graph for F_Enhanced saved to data/graphml/F_Enhanced.graphml
Graph for S_Enhanced saved to data/graphml/S_Enhanced.graphml
Graph for FS_Enhanced saved to data/graphml/FS_Enhanced.graphml
Graph for HM_Suppressed saved to data/graphml/HM_Suppressed.graphml
Graph for Neu_Suppressed saved to data/graphml/Neu_Suppressed.graphml
Graph for F_Suppressed saved to data/graphml/F_Suppressed.graphml
Graph for S_Suppressed saved to data/graphml/S_Suppressed.graphml
Graph for FS_Suppressed saved to data/graphml/FS_Suppressed.graphml
Total graph saved to data/graphml/Total.graphml


In [72]:
import os
import networkx as nx
from pyvis.network import Network

def create_html(path):
    # Ensure the file exists
    if not os.path.exists(path):
        print(f"File does not exist: {path}")
        return

    try:
        # Load CYS data from GraphML file
        graph = nx.read_graphml(path)
        # Extracting and displaying the node metadata
        node_metadata = {node: graph.nodes[node] for node in graph.nodes()}
        node_metadata_list = list(node_metadata.items())[:10]  # Displaying metadata for first 10 nodes for brevity
        print("Node metadata (first 10):", node_metadata_list)
        
        # Create a Pyvis network
        net = Network(notebook=False, height="750px", width="100%")

        # Node customization settings
        special_nodes = ['BSG', 'CD44', 'EGFR', 'SLC3A2']
        special_color = '#FE81B8'  # Example color (red)
        font_size = 14  # Adjust font size
        #print(graph.nodes)
        for node, attrs in graph.nodes(data=True):
            if node in special_nodes:
                net.add_node(node, label=attrs.get('id', ''), shape='diamond', size=20, font=dict(size=font_size))
            else:
                net.add_node(node, label=attrs.get('id', ''), size=12, font=dict(size=font_size), color=special_color)

        # Add edges with hover data
        for source, target, attrs in graph.edges(data=True):
            title = f"{source}_{target}"
            net.add_edge(source, target, title=title)

        # Generate the HTML file path
        html_path = path.replace('.graphml', '.html').replace('graphml', 'Total_html')
        print(f"Saving HTML to {html_path}")

        # Save the network to an HTML file
        net.write_html(html_path)
        print(f"HTML file saved to {html_path}")

    except Exception as e:
        print(f"Error processing file {path}: {e}")

In [73]:
# Iterate over all GraphML files in the directory and create HTML files
for file in os.listdir('data/graphml/'):
    if file.endswith('.graphml'):
        create_html(os.path.join('data/graphml', file))

Node metadata (first 10): [('BSG', {}), ('ADAM9', {}), ('CPT1A', {}), ('LGALS3', {}), ('CD44', {}), ('EPCAM', {}), ('ITGB1', {}), ('NDUFS1', {}), ('PFKP', {}), ('S100A8', {})]
Saving HTML to data/Total_html/Neu_Enhanced.html
HTML file saved to data/Total_html/Neu_Enhanced.html
Node metadata (first 10): [('BSG', {}), ('ADAM9', {}), ('ARF4', {}), ('ARF5', {}), ('ARF6', {}), ('ATP2A2', {}), ('ATP2B1', {}), ('ATP5F1A', {}), ('ATP5F1B', {}), ('ATP5PO', {})]
Saving HTML to data/Total_html/total.html
HTML file saved to data/Total_html/total.html
Node metadata (first 10): [('BSG', {}), ('ARF5', {}), ('ATP5F1A', {}), ('ATP5PO', {}), ('ATP6V1H', {}), ('CCT3', {}), ('ESYT2', {}), ('HAX1', {}), ('LPCAT1', {}), ('PHB1', {})]
Saving HTML to data/Total_html/Independent.html
HTML file saved to data/Total_html/Independent.html
Node metadata (first 10): [('BSG', {}), ('SACM1L', {}), ('CD44', {}), ('ALDH1A3', {}), ('NDUFS1', {}), ('S100A8', {}), ('S100A9', {}), ('SPINT2', {}), ('EGFR', {}), ('AP1M1', {})

## make plot

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
# read data with two level column name 
df = pd.read_csv('data/Total_Network_GlycanList.csv', header=[0,1])
df

Pair    Bait     Prey  BaitID  PreyID   HM               Und  \
             Pair    Bait     Prey  BaitID  PreyID    1    2    3       1   
0      EGFR_ACIN1    EGFR    ACIN1  P00533  Q9UKV3  100  100  100   32.35   
1       BSG_ADAM9     BSG    ADAM9  P35613  Q13443  100  100  100  107.78   
2      EGFR_ADAM9    EGFR    ADAM9  P00533  Q13443  100  100  100   97.76   
3     CD44_AGPAT2    CD44   AGPAT2  P16070  O15120  100  100  100   55.60   
4    CD44_ALDH1A3    CD44  ALDH1A3  P16070  P47895  100  100  100   93.29   
..            ...     ...      ...     ...     ...  ...  ...  ...     ...   
151  SLC3A2_VDAC1  SLC3A2    VDAC1  P08195  P21796  100  100  100  133.07   
152       BSG_VIM     BSG      VIM  P35613  P08670  100  100  100   32.32   
153      BSG_VMP1     BSG     VMP1  P35613  Q96GC9  100  100  100   32.27   
154       BSG_WLS     BSG      WLS  P35613  Q5T9L3  100  100  100   65.35   
155     EGFR_XP32    EGFR     XP32  P00533  Q5T750  100  100  100  331.09   

                        Fuc                     Sia                  SiaFuc  \
          2       3       1       2       3       1       2       3       1   
0     34.65   32.26   20.18   38.10   28.21   36.26   31.04   34.72   29.62   
1    122.96  117.47   67.87   54.14   67.54  103.45   86.37   96.52   77.55   
2     87.40   76.19   84.20   59.90   59.02   51.61   55.25   41.59   49.61   
3     61.06   43.19   41.77   33.41   34.09   36.85   28.62   37.63   68.21   
4     77.61  100.98   71.77   76.95   71.96   49.15   37.26   40.76  129.09   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
151  112.75   90.68   98.14  106.23   89.65  192.79  210.26  144.99  239.33   
152   25.84   21.02   77.93   53.66   56.53   37.42   36.92   29.27   36.23   
153   42.22   30.05   23.86   43.64   33.04   24.87   40.88   32.76   21.59   
154   59.23   65.03   46.34   36.49   36.99   51.38   46.20   50.13   55.33   
155  224.96  299.76  154.22  160.28  137.84  301.89  242.72  253.33  214.24   

                     
          2       3  
0     25.55   32.16  
1     75.73   52.59  
2     52.84   53.85  
3     71.81   48.74  
4    125.91   84.88  
..      ...     ...  
151  211.69  254.12  
152   35.14   24.53  
153   31.64   24.46  
154   59.17   52.98  
155  219.19  139.27  

[156 rows x 20 columns]

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns    


In [7]:
df1 = df.iloc[:, [0] + list(range(5, 20))]
df1

Pair   HM               Und                     Fuc          \
             Pair    1    2    3       1       2       3       1       2   
0      EGFR_ACIN1  100  100  100   32.35   34.65   32.26   20.18   38.10   
1       BSG_ADAM9  100  100  100  107.78  122.96  117.47   67.87   54.14   
2      EGFR_ADAM9  100  100  100   97.76   87.40   76.19   84.20   59.90   
3     CD44_AGPAT2  100  100  100   55.60   61.06   43.19   41.77   33.41   
4    CD44_ALDH1A3  100  100  100   93.29   77.61  100.98   71.77   76.95   
..            ...  ...  ...  ...     ...     ...     ...     ...     ...   
151  SLC3A2_VDAC1  100  100  100  133.07  112.75   90.68   98.14  106.23   
152       BSG_VIM  100  100  100   32.32   25.84   21.02   77.93   53.66   
153      BSG_VMP1  100  100  100   32.27   42.22   30.05   23.86   43.64   
154       BSG_WLS  100  100  100   65.35   59.23   65.03   46.34   36.49   
155     EGFR_XP32  100  100  100  331.09  224.96  299.76  154.22  160.28   

                Sia                  SiaFuc                  
          3       1       2       3       1       2       3  
0     28.21   36.26   31.04   34.72   29.62   25.55   32.16  
1     67.54  103.45   86.37   96.52   77.55   75.73   52.59  
2     59.02   51.61   55.25   41.59   49.61   52.84   53.85  
3     34.09   36.85   28.62   37.63   68.21   71.81   48.74  
4     71.96   49.15   37.26   40.76  129.09  125.91   84.88  
..      ...     ...     ...     ...     ...     ...     ...  
151   89.65  192.79  210.26  144.99  239.33  211.69  254.12  
152   56.53   37.42   36.92   29.27   36.23   35.14   24.53  
153   33.04   24.87   40.88   32.76   21.59   31.64   24.46  
154   36.99   51.38   46.20   50.13   55.33   59.17   52.98  
155  137.84  301.89  242.72  253.33  214.24  219.19  139.27  

[156 rows x 16 columns]

In [18]:
for index, row in df1.iterrows():
    # Extracting the 'Pair' name and data for the row
    pair_name = row['Pair'].item()
    data = row[1:].astype(float)  # Convert data to float
    #conditions_unique = set([x[0] for x in data.index ])
    conditions_unique = ["HM","Und","Fuc","Sia","SiaFuc"]
    
    # Prepare data for plotting
    plot_data = pd.DataFrame(columns=conditions_unique)
    for condition in conditions_unique:
        # Collect data for each condition
        condition_data = data[condition]
        plot_data[condition] = condition_data.values
    #print(plot_data)
    
    # Normalize data if needed, for example to percentage
    #plot_data = plot_data / plot_data.sum() * 100
    
    # Plotting
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=plot_data,palette=['#B14743', '#D48640', '#539045', '#8E73AE', '#44729D'],saturation=1)
    plt.title(pair_name)
    plt.ylabel('Normalized Abundance (%)')
    
    # Save the plot as a PNG file
    plt.show()
    # use great resolution for publication
    plt.savefig(f"data/boxplot/{pair_name}.png", dpi=800)
    plt.close()  # Close the plot to avoid displaying it in the notebook
    



/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/2z/1x5j42m17l3f8ksd7yv0jgv00000gn/T/ipykernel_94378/1010801694.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be